# Lab 1: Building Effective Prompts with the CRAFT Framework

In this hands-on lab, you'll learn to systematically construct high-quality prompts using the CRAFT framework. You'll evaluate prompt quality using a manual scoring rubric, diagnose flawed prompts, and build reusable templates — all by working directly with an AI tool (ChatGPT, Claude, etc.).

## Learning Objectives
- Apply the CRAFT framework systematically to any business task
- Diagnose and fix flawed prompts using a structured rubric
- Critically evaluate when a prompt "looks good" but fails in practice
- Build reusable prompt templates with placeholders
- Adapt prompts for different audiences and stakeholders

**Duration:** 55–65 minutes | **Difficulty:** Beginner to Intermediate

**What You Need:** Access to an AI tool (ChatGPT, Claude, Copilot, or similar)

---

## Part 1: The CRAFT Framework

The CRAFT framework breaks every prompt into five essential components:

| Component | What It Does | Example |
|-----------|-------------|---------||
| **C** — Context | Background information the AI needs | "Our hospital network is evaluating AI tools..." |
| **R** — Role | The expert persona the AI should adopt | "Act as a healthcare technology analyst..." |
| **A** — Action | The specific task you want performed | "Write a 600-word briefing comparing three solutions..." |
| **F** — Format | How the output should be structured | "Use bullet points for findings, then a numbered list..." |
| **T** — Tone | The voice and style of the response | "Professional, evidence-based, and persuasive" |

### CRAFT Scoring Rubric

Use this rubric to evaluate any prompt on a 1–5 scale across five quality dimensions:

| Dimension | 1 (Poor) | 2 (Weak) | 3 (Adequate) | 4 (Good) | 5 (Excellent) |
|-----------|----------|----------|---------------|----------|---------------|
| **Clarity** | Ambiguous, multiple interpretations | Somewhat clear but key details missing | Main request is clear | Clear with minor ambiguities | Crystal clear, single interpretation |
| **Specificity** | No concrete details | One or two details | Some details but gaps | Most details included | Precise numbers, names, constraints |
| **Context** | No background given | Minimal context | Some context provided | Good context with audience | Rich context with situation and audience |
| **Format** | No format guidance | Vague format request | Basic format specified | Detailed structure requested | Exact structure with lengths and sections |
| **Actionability** | No clear action | Vague action verb | Clear action but broad | Specific action with scope | Precise, measurable action with deliverable |

**How to score:** Read the prompt, then rate each dimension 1–5. Add up for a total out of 25.

---

## Before & After: The Power of CRAFT

Read each "Before" prompt below. Pick one and score it using the rubric above — **before** looking at the improved version.

---

### Pair 1: "Write about AI"

**BEFORE:**
> Write about AI

---

### Pair 2: "Summarize this"

**BEFORE:**
> Summarize this

---

### Pair 3: "Help with email"

**BEFORE:**
> Help with email

---

**Pick one pair and score the BEFORE prompt using the rubric above.**

### Your BEFORE Score

**Which pair did you choose?** _______________________________________________

| Dimension | Score (1-5) |
|-----------|-------------|
| Clarity | ___ |
| Specificity | ___ |
| Context | ___ |
| Format | ___ |
| Actionability | ___ |
| **TOTAL** | ___/25 |

---

### Now See the AFTER Versions

Here are the same prompts rewritten using the CRAFT framework. Find the pair you scored above and score the AFTER version.

---

### Pair 1: "Write about AI" — AFTER (with CRAFT)

> **Context:** Our hospital network is evaluating AI tools to reduce diagnostic errors. The audience is the clinical leadership team.
> 
> **Role:** Act as a healthcare technology analyst with expertise in clinical AI.
> 
> **Action:** Write a 600-word briefing on how AI-assisted diagnostics are reducing misdiagnosis rates, including 3 specific case studies from peer-reviewed research.
> 
> **Format:** Start with a 2-sentence executive summary, then use bullet points for each case study, and end with a numbered list of 3 recommendations.
> 
> **Tone:** Professional, evidence-based, and persuasive.

---

### Pair 2: "Summarize this" — AFTER (with CRAFT)

> **Context:** The attached 40-page quarterly earnings report covers revenue, operating costs, and forward guidance for Q3 2025.
> 
> **Role:** Act as a senior financial analyst preparing materials for the CFO.
> 
> **Action:** Summarize the report into an executive briefing that highlights the 5 most critical data points, identifies 2 risks, and notes any upward or downward trends.
> 
> **Format:** One heading per section, bullet points for data, and a comparison table for quarter-over-quarter metrics. Keep under 300 words.
> 
> **Tone:** Concise, analytical, and neutral.

---

### Pair 3: "Help with email" — AFTER (with CRAFT)

> **Context:** We delivered a software demo to a prospective enterprise client last Tuesday. They asked about SSO integration and data residency in the EU. We need to follow up within 48 hours.
> 
> **Role:** Act as an enterprise account executive at a B2B SaaS company.
> 
> **Action:** Draft a follow-up email that thanks the client for their time, directly addresses their SSO and data-residency questions, and proposes a 30-minute technical deep-dive call next week.
> 
> **Format:** Subject line + body. Body has a greeting, 3 short paragraphs, and a clear call-to-action with 2 proposed meeting times.
> 
> **Tone:** Warm, confident, and professional.

---

**Now score the AFTER version of your chosen pair.**

### Your AFTER Score

| Dimension | Score (1-5) |
|-----------|-------------|
| Clarity | ___ |
| Specificity | ___ |
| Context | ___ |
| Format | ___ |
| Actionability | ___ |
| **TOTAL** | ___/25 |

**How much did the score improve?** _______________________________________________

---

<!-- submit-exercise -->

## Exercise 1: Diagnose Broken Prompts (5 minutes)

Below are three prompts that have all five CRAFT components filled in — they *look* complete. But each has a subtle flaw that would make it ineffective in practice. Your job:

1. **Read** each prompt carefully
2. **Identify** the specific flaw (use the hint in the table)
3. **Write** a corrected version

| Prompt | Flaw Type | Hint |
|--------|-----------|------|
| Broken 1 | Context / Role mismatch | The role doesn't match what the context needs |
| Broken 2 | Vague action | The action verb is too ambiguous to produce useful output |
| Broken 3 | Wrong tone for audience | The tone will alienate the target reader |

---

### Broken Prompt 1: Context / Role Mismatch

> **Context:** Our hospital is evaluating whether to adopt an AI-powered radiology screening tool. The decision committee includes the Chief Medical Officer and head of IT.
> 
> **Role:** Act as a social media influencer specializing in lifestyle content.
> 
> **Action:** Write a 400-word recommendation memo evaluating the clinical accuracy, integration requirements, and ROI of the AI tool.
> 
> **Format:** Executive memo with sections: Summary, Clinical Evidence, Technical Requirements, Cost-Benefit Analysis.
> 
> **Tone:** Professional, evidence-based, and balanced.

**What's wrong?** _______________________________________________

**Your corrected Role:** _______________________________________________

### Broken Prompt 2: Vague Action

> **Context:** Our e-commerce company had a 15% cart abandonment increase last quarter. The product team needs actionable insights from user session data covering 50,000 transactions.
> 
> **Role:** Act as a senior UX researcher with expertise in e-commerce analytics.
> 
> **Action:** Look at the data and tell us what you think about it.
> 
> **Format:** Structured report with numbered findings, each including the data pattern, its likely cause, and a specific fix.
> 
> **Tone:** Analytical, direct, and data-driven.

**What's wrong?** _______________________________________________

**Your corrected Action:** _______________________________________________

### Broken Prompt 3: Wrong Tone for Audience

> **Context:** A Fortune 500 CEO is presenting to the board of directors about the company's AI transformation roadmap for 2026. Board members include institutional investors and industry veterans.
> 
> **Role:** Act as a management consultant at a top-tier strategy firm.
> 
> **Action:** Write the opening remarks for the board presentation, covering strategic rationale, expected ROI, and competitive positioning.
> 
> **Format:** Speech script, 300 words, with clear transitions between the three topics.
> 
> **Tone:** Super casual, lots of slang, use emojis and exclamation marks!!!

**What's wrong?** _______________________________________________

**Your corrected Tone:** _______________________________________________

---

<!-- submit-exercise -->

## Part 2: Prompt Templates

Instead of crafting every prompt from scratch, experienced prompt engineers maintain a **template library** — reusable CRAFT prompts with `{placeholder}` values that can be filled in for different situations.

### Example Templates

**Template 1: Email Writer**
> **Context:** We are a {company_type} company. The recipient is {recipient}.
> 
> **Role:** Act as a professional communications specialist.
> 
> **Action:** Draft a {email_type} email about {topic} that includes a clear call-to-action.
> 
> **Format:** Subject line + 3 concise paragraphs. Keep under 200 words.
> 
> **Tone:** {tone}

**Template 2: Meeting Summarizer**
> **Context:** The meeting was a {meeting_type} attended by {attendees}. Duration: {duration}.
> 
> **Role:** Act as an executive assistant skilled at distilling meeting notes.
> 
> **Action:** Summarize the meeting into key decisions, action items with owners, and open questions. Flag any items that are blocked.
> 
> **Format:** Headings: Decisions, Action Items (table with Owner and Due Date), Open Questions. Under 250 words.
> 
> **Tone:** Neutral and precise.

**Template 3: Data Analyst**
> **Context:** The dataset contains {dataset_description}. The stakeholder is {audience}.
> 
> **Role:** Act as a senior data analyst at a {industry} company.
> 
> **Action:** Analyze the data to identify the top {num_insights} insights, highlight anomalies, and recommend next steps.
> 
> **Format:** Executive summary (3 sentences), bullet points per insight, anomaly table with severity ratings.
> 
> **Tone:** Data-driven and actionable.

**Key idea:** Templates let you standardize quality while customizing for each use case. The `{placeholders}` are the only things that change.

---

## Exercise 2: Game the Rubric (10 minutes)

The CRAFT scoring rubric is a useful tool, but no rubric is perfect. In this exercise, you'll test its limits.

**Before you start,** study the worked examples below to see what "gaming the rubric" looks like in practice. Then choose your scoring rubric and write your own prompts in Part A and Part B.

**Part A — High-scoring but terrible prompt:** Write a prompt that would score **20+ out of 25** on the rubric but would produce **useless output** from an AI. Pack it with all the right structural elements while making the actual request contradictory, impossible, or incoherent.

**Part B — Low-scoring but effective prompt:** Write a prompt that would score **10 or lower** on the rubric but would actually produce **excellent, useful output** from a real AI. Keep it short and direct without the structural elements the rubric rewards.

**Part C — Test both prompts** in your AI tool and paste the results.

---

### Worked Examples: What "Gaming the Rubric" Looks Like

Before you write your own, study these two examples. Notice how rubric scores can be misleading.

**These are examples for reference — your own prompts go in Part A and Part B below.**

---

#### Example 1: High Score, Terrible Prompt (Score: 21/25)

> **Context:** Our multinational retail corporation operates in 14 countries and serves 2.3 million customers annually. The executive leadership team is preparing for the Q4 board review.
>
> **Role:** Act as a senior strategy consultant specializing in retail operations, supply chain optimization, and consumer psychology.
>
> **Action:** Write a 75-word report that includes a full competitive analysis of 15 major retailers, a 10-year demand forecast model for each of our 14 markets, a complete organizational restructuring plan with role-by-role headcount changes, and a customer sentiment analysis based on 500,000 survey responses.
>
> **Format:** Executive summary, 4 comparison tables, a numbered action plan with 25 items, and a risk matrix. Use bullet points for all findings.
>
> **Tone:** Authoritative, data-driven, and concise.

| Dimension | Score | Why It Scores Well |
|-----------|-------|--------------------|
| Clarity | 4 | Language is precise and professional |
| Specificity | 5 | Packed with exact numbers (14 countries, 15 retailers, 500K surveys) |
| Context | 4 | Rich organizational context with audience identified |
| Format | 4 | Multiple format elements requested |
| Actionability | 4 | Clear action verbs with defined deliverables |
| **TOTAL** | **21/25** | |

**Why it fails:** The prompt demands content that would fill 50+ pages crammed into 75 words. The constraints are physically contradictory — you cannot fit 15 competitive analyses, 14 forecast models, a restructuring plan, and a sentiment analysis into 75 words. An AI will either ignore the word limit or produce meaningless fragments. **The rubric rewards the *appearance* of structure without checking whether the constraints are achievable.**

---

#### Example 2: Low Score, Effective Prompt (Score: 7/25)

> Compare the pros and cons of microservices vs monolith for a 10-person startup.

| Dimension | Score | Why It Scores Low |
|-----------|-------|-------------------|
| Clarity | 2 | Short, no elaboration |
| Specificity | 2 | Only one concrete detail ("10-person startup") |
| Context | 1 | No background, no audience, no industry |
| Format | 1 | No format specified at all |
| Actionability | 1 | Simple question, no defined deliverable |
| **TOTAL** | **7/25** | |

**Why it works:** The question is inherently focused — "pros and cons" constrains the structure, "microservices vs monolith" is a well-defined comparison, and "10-person startup" provides the key constraint. The AI will naturally produce a balanced comparison because the question implies one. **Sometimes a direct question with the right specificity outperforms an elaborately structured prompt.**

---

### Choose Your Scoring Rubric

You can use the **standard CRAFT rubric** from Part 1 (Clarity, Specificity, Context, Format, Actionability — each scored 1-5), or define your own dimensions below.

**Option A: Use the Standard CRAFT Rubric** — Skip ahead to Part A. The scoring tables already use CRAFT dimensions.

**Option B: Define Your Own Rubric** — If you think the CRAFT rubric has blind spots, design your own 5-dimension rubric here. Use these dimensions instead when scoring in Part A and Part B.

| # | Your Dimension | What It Measures (1 sentence) |
|---|----------------|-------------------------------|
| 1 | _______________ | _______________ |
| 2 | _______________ | _______________ |
| 3 | _______________ | _______________ |
| 4 | _______________ | _______________ |
| 5 | _______________ | _______________ |

**Tip:** Good alternative dimensions might include *Feasibility* (can the AI actually do this?), *Coherence* (do the parts work together?), or *Efficiency* (is the prompt as short as possible without losing quality?).

---

### Part A: High-Scoring Terrible Prompt (target: 20+ / 25)

**Your prompt:**

> _______________________________________________
> _______________________________________________
> _______________________________________________
> _______________________________________________

**Your rubric score:**

| Dimension | Score (1-5) | Why it scores well |
|-----------|-------------|-------------------|
| Clarity | ___ | |
| Specificity | ___ | |
| Context | ___ | |
| Format | ___ | |
| Actionability | ___ | |
| **TOTAL** | ___/25 | |

**Paste the AI output here:**

> _______________________________________________

**Why is this output useless despite the high score?**

_______________________________________________

### Part B: Low-Scoring Effective Prompt (target: 10 or lower)

**Your prompt:**

> _______________________________________________

**Your rubric score:**

| Dimension | Score (1-5) | Why it scores low |
|-----------|-------------|------------------|
| Clarity | ___ | |
| Specificity | ___ | |
| Context | ___ | |
| Format | ___ | |
| Actionability | ___ | |
| **TOTAL** | ___/25 | |

**Paste the AI output here:**

> _______________________________________________

**Why is this output useful despite the low score?**

_______________________________________________

### Part C: Reflection

**What are 2 blind spots in the rubric that your examples expose?**

1. _______________________________________________

2. _______________________________________________

---

<!-- submit-exercise -->

## Exercise 3: Build a Template from Scratch (15 minutes)

Design a **competitive analysis** prompt template from scratch.

**Requirements:**
1. Use the CRAFT framework with **at least 4 placeholders** (e.g., `{company}`, `{competitor}`, `{industry}`, `{metric}`)
2. Fill the template for **2 different industries** (e.g., fintech vs. healthcare)
3. **Test both filled prompts** in your AI tool
4. Score both using the rubric — aim for **20+ out of 25** each
5. Compare the quality of the two AI outputs

---

### Your Competitive Analysis Template

> **Context:** _______________________________________________
> 
> **Role:** _______________________________________________
> 
> **Action:** _______________________________________________
> 
> **Format:** _______________________________________________
> 
> **Tone:** _______________________________________________

**Placeholders used:** {___}, {___}, {___}, {___}

### Industry 1: _______________

**Filled prompt (replace all placeholders):**

> _______________________________________________
> _______________________________________________
> _______________________________________________
> _______________________________________________
> _______________________________________________

**Paste AI output here:**

> _______________________________________________

**Score:** ___/25

### Industry 2: _______________

**Filled prompt (replace all placeholders):**

> _______________________________________________
> _______________________________________________
> _______________________________________________
> _______________________________________________
> _______________________________________________

**Paste AI output here:**

> _______________________________________________

**Score:** ___/25

### Comparison

| Dimension | Industry 1 Score | Industry 2 Score | Notes |
|-----------|-----------------|-----------------|-------|
| Clarity | ___ | ___ | |
| Specificity | ___ | ___ | |
| Context | ___ | ___ | |
| Format | ___ | ___ | |
| Actionability | ___ | ___ | |
| **TOTAL** | ___/25 | ___/25 | |

**Which industry produced the better output, and why?**

_______________________________________________

---

<!-- submit-exercise -->

## Exercise 4: The Audience Pivot (15 minutes)

The same information needs radically different prompts depending on who will read the output. Write **three CRAFT prompts** that all request the same core information — a summary of your company's Q3 AI initiative results — but tailored for three different audiences:

| Audience | What They Care About | Expected Tone |
|----------|---------------------|---------------|
| **Engineering team** | Technical details, architecture, performance metrics | Technical, precise |
| **VP of Product** | Roadmap impact, resource needs, timelines | Strategic, concise |
| **CEO** | Business value, competitive advantage, ROI | Executive, high-level |

**Instructions:**
1. Write all 3 prompts below
2. Test all 3 in your AI tool
3. Fill in the comparison table
4. Reflect on which CRAFT component changes the most across audiences

**Getting started:** Default prompts are provided below as starting points. You can use them as-is, modify them, or replace them entirely with your own.

---

### Prompt for Engineering Team

> **Context:** Our company deployed 3 AI features in Q3: an ML-powered search ranking system, a document classification pipeline, and a customer support chatbot. The engineering team of 25 needs a technical debrief covering system architecture, model performance, and infrastructure costs.
>
> **Role:** Act as a principal engineer who led the Q3 AI initiative.
>
> **Action:** Write a technical summary of all 3 AI features covering: architecture decisions (tech stack, model selection, deployment infrastructure), performance metrics (latency p50/p99, accuracy, throughput), and lessons learned including what you would change.
>
> **Format:** One section per feature with subheadings for Architecture, Metrics, and Lessons Learned. Include a comparison table of model performance across all 3 features. Use code-style formatting for technical terms.
>
> **Tone:** Technical, precise, and candid about trade-offs.

**Paste AI output here:**

> _______________________________________________

### Prompt for VP of Product

> **Context:** Our company shipped 3 AI features in Q3 as part of the product roadmap. The VP of Product needs to understand how these features affect the Q4 roadmap, what resources are needed to scale them, and which features should be prioritized.
>
> **Role:** Act as a product manager reporting on the Q3 AI initiative.
>
> **Action:** Write a product update that covers: feature adoption metrics (DAU, conversion lift, NPS impact), roadmap implications for Q4 (what shipped on time, what slipped, dependencies), and resource requests (headcount, infrastructure budget, vendor costs).
>
> **Format:** Executive summary (3 bullets), then a table with columns: Feature, Status, Q4 Priority, Resource Ask. Follow with a short "Risks & Dependencies" section.
>
> **Tone:** Strategic, concise, and focused on decisions the VP needs to make.

**Paste AI output here:**

> _______________________________________________

### Prompt for CEO

> **Context:** Our company invested $2.4M in AI initiatives in Q3, deploying 3 customer-facing AI features. The CEO is preparing talking points for the next investor call and needs to understand business impact and competitive positioning.
>
> **Role:** Act as the Chief Strategy Officer briefing the CEO.
>
> **Action:** Write a one-page executive brief covering: business impact (revenue influenced, cost savings, customer satisfaction gains), competitive advantage (how these features differentiate us in the market), and investment outlook (Q4 budget request with projected ROI).
>
> **Format:** 3 sections with bold headers. Lead each section with a single headline metric. No technical jargon. Keep under 400 words total.
>
> **Tone:** Executive, high-level, and persuasive with emphasis on business value.

**Paste AI output here:**

> _______________________________________________

### Comparison Table

| CRAFT Dimension | Engineering | VP of Product | CEO | Which Changed Most? |
|----------------|-------------|---------------|-----|-------------------|
| Context | | | | |
| Role | | | | |
| Action | | | | |
| Format | | | | |
| Tone | | | | |

**Which CRAFT component changes the most across audiences, and why?**

_______________________________________________

**Which audience was hardest to write for, and why?**

_______________________________________________

---

<!-- submit-exercise -->

## Exercise 5: The Prompt Audit (10 minutes)

Below are three prompts that all *look* professional and well-structured. But each has a hidden practical problem that would produce poor results from an AI. These problems won't show up in a rubric score — they require real-world judgment to catch.

**Your task:**
1. Read each prompt carefully
2. Identify the hidden real-world problem
3. Rank the three prompts from **most effective** to **least effective** in practice
4. Write a 2–3 sentence justification for your ranking

**Possible problem types:** conflicting instructions, impossible constraints, missing critical context, overspecification, scope creep

---

### Audit Prompt A

> **Context:** Our company is launching a new product in the healthcare industry targeting hospital administrators and clinical stakeholders.
> 
> **Role:** Act as a senior marketing strategist with 15 years experience.
> 
> **Action:** Write a **100-word** executive summary that comprehensively covers all 12 market segments, including 3 detailed case studies with full financial breakdowns for each, a competitive landscape analysis of the top 8 competitors, and a 5-year revenue projection model.
> 
> **Format:** Use bullet points, numbered lists, and a comparison table.
> 
> **Tone:** Concise and data-driven.

**Hidden problem:** _______________________________________________

### Audit Prompt B

> **Context:** We are a fintech company preparing for a board meeting. The audience is our board of directors and key stakeholders.
> 
> **Role:** Act as a financial analyst AND a product designer AND a legal compliance officer AND a marketing strategist.
> 
> **Action:** Write a board presentation that covers Q3 financial results, redesign the customer onboarding flow, draft new compliance policies for 5 regions, create a marketing campaign for the next product launch, and build a competitive analysis of the market sector.
> 
> **Format:** Structured report with numbered sections, bullet points, and a summary table. Include 10 items per section.
> 
> **Tone:** Professional, precise, and actionable.

**Hidden problem:** _______________________________________________

### Audit Prompt C

> **Context:** Based on the data from last quarter, our company needs to make a strategic decision. The team is evaluating multiple options in our industry sector.
> 
> **Role:** Act as a management consultant at a top-tier strategy firm specializing in our specific market and customer segment.
> 
> **Action:** Analyze the situation and recommend which option we should pursue, including a comparison table of 3 alternatives with estimated ROI for each stakeholder group.
> 
> **Format:** Executive summary (5 sentences), bullet points for each option, numbered list of 3 recommendations.
> 
> **Tone:** Authoritative, data-driven, and actionable.

**Hidden problem:** _______________________________________________

### Your Ranking (most effective to least effective)

| Rank | Prompt | Justification |
|------|--------|--------------|
| 1 (Best) | ___ | |
| 2 | ___ | |
| 3 (Worst) | ___ | |

**Your 2–3 sentence justification:**

_______________________________________________

---

<!-- submit-exercise -->